In [1]:
pip install spotipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:

import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id="315a0b8d76314d19b3b2de3f13f8d2b8",
        client_secret="14d51a4c5ce7407eb4657a62239d0981",
        redirect_uri="http://127.0.0.1:8888/callback",
        scope="playlist-read-private playlist-modify-private"
        
))

me = sp.current_user()
print("Logged in as:", me["display_name"])


Logged in as: 지랄


/Users/donghyunhahn/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# reading the url of the playlist from user input
playlist_url = input("Playlist URL: ").strip()
playlist_id = playlist_url.split("/")[-1].split("?")[0]

print("Playlist ID:", playlist_id)

results = sp.playlist_items(playlist_id)
for item in results["items"]:
    track = item["track"]
    if track:
        name = track["name"]
        artists = ", ".join(a["name"] for a in track["artists"])
        print(name, "-", artists)


Playlist ID: 7w0WuNaAhod1MLERVfiXgh
It Ain't Necessarily So - Mary Lou Williams
When Sunny Gets Blue - McCoy Tyner
Love Theme From Spartacus - Rudy Van Gelder Remaster / 2006 - Yusef Lateef
The Shadow of Your Smile - Dexter Gordon, Slide Hampton
So Tender - Keith Jarrett, Gary Peacock, Jack DeJohnette
In A Sentimental Mood - Duke Ellington, John Coltrane
Quiet Temple - Donald Byrd & Booker Little
Dancing - bLAck pARty
The Things We Did Last Summer - Freddie Hubbard
San Juan Sunset - Deodato
Stardust - Bobby Hutcherson
Crystal Silence - Chick Corea
Rosita - Coleman Hawkins, Ben Webster
Blue World - John Coltrane
The Awakening - Ahmad Jamal Trio
Alone Together - Chet Baker
Petite Belle - Art Farmer
How Deep Is The Ocean - John Coltrane, Hank Mobley, Al Cohn, Zoot Sims
Isfahan - Duke Ellington
Trust - Roy Hargrove
My Foolish Heart - Bill Evans Trio
A Taste of Honey - Live - Paul Desmond
Quiet Nights Of Quiet Stars (Corcovado) - Oscar Peterson Trio
Maybe Tomorrow - Grant Green


In [5]:
# collecting track IDs and basic info
tracks = []
for item in results["items"]:
    track = item["track"]
    if not track:
        continue
    tracks.append(
        {
            "id": track["id"],
            "name": track["name"],
            "artists": ", ".join(a["name"] for a in track["artists"]),
        }
    )

print("Total tracks collected:", len(tracks))
print("Sample IDs:", [t["id"] for t in tracks[:5]])



Total tracks collected: 24
Sample IDs: ['2qswPLZD9CbUoSidGNyxwm', '1J9iVCaEriyoMXpj2XszhU', '7yzM0pEseQUpb6BMViOPEA', '0sRAziWSeUedBSKjapsy9F', '0Dt1B5dNfajRczQKZA87aL']


In [7]:
# collect unique artist IDs
artist_ids = set()
for item in results["items"]:
    track = item["track"]
    if not track:
        continue
    for a in track["artists"]:
        artist_ids.add(a["id"])

artist_ids = list(artist_ids)
print("Unique artists:", len(artist_ids))

# batch get artist info
artists_info = {}
for i in range(0, len(artist_ids), 50):
    chunk = artist_ids[i:i+50]
    resp = sp.artists(chunk)
    for art in resp["artists"]:
        artists_info[art["id"]] = {
            "name": art["name"],
            "genres": art["genres"],
            "popularity": art["popularity"],
        }

# attach main genre to each track
for t in tracks:
    # pick first artist’s genres
    first_artist_id = next(a["id"] for a in results["items"][0]["track"]["artists"])


Unique artists: 29


In [9]:
# 이미 앞에서 만든 부분 (참고용)
# results = sp.playlist_items(playlist_id)

tracks = []
artist_ids = set()

for item in results["items"]:
    track = item["track"]
    if not track:
        continue
    
    # 트랙 기본 정보
    t_id = track["id"]
    t_name = track["name"]
    t_artists = track["artists"]
    
    tracks.append(
        {
            "id": t_id,
            "name": t_name,
            "artists": ", ".join(a["name"] for a in t_artists),
            "artist_ids": [a["id"] for a in t_artists],
        }
    )
    
    # 아티스트 ID 모으기
    for a in t_artists:
        if a["id"]:
            artist_ids.add(a["id"])

artist_ids = list(artist_ids)
print("Total tracks:", len(tracks))
print("Unique artists:", len(artist_ids))


Total tracks: 24
Unique artists: 29


In [10]:
artists_info = {}

# 50개씩 배치로 호출
for i in range(0, len(artist_ids), 50):
    chunk = artist_ids[i:i+50]
    resp = sp.artists(chunk)  # 여기가 /v1/artists 호출 [web:78][web:53]
    
    for art in resp["artists"]:
        artists_info[art["id"]] = {
            "name": art["name"],
            "genres": art["genres"],          # 리스트
            "popularity": art["popularity"],  # 0~100
        }

print("Fetched artist info for:", len(artists_info), "artists")


Fetched artist info for: 29 artists


In [11]:
def mood_from_genres(genres):
    if not genres:
        return "unknown"
    
    g = " ".join(genres).lower()
    
    if any(x in g for x in ["lo-fi", "lofi", "chill", "ambient", "study"]):
        return "chill"
    if any(x in g for x in ["hip hop", "rap", "trap", "drill"]):
        return "hype"
    if any(x in g for x in ["r&b", "soul", "ballad"]):
        return "emotional"
    if any(x in g for x in ["edm", "house", "techno", "dance"]):
        return "dance"
    if any(x in g for x in ["jazz", "blues"]):
        return "smooth"
    if any(x in g for x in ["metal", "hardcore"]):
        return "intense"
    
    return "mixed"


In [12]:
from collections import Counter

playlist_moods = []
genre_counter = Counter()

for t in tracks:
    artist_id = t["artist_ids"][0] if t["artist_ids"] else None
    artist_data = artists_info.get(artist_id, {})
    genres = artist_data.get("genres", [])
    popularity = artist_data.get("popularity", None)
    
    mood = mood_from_genres(genres)
    
    t["genres"] = genres
    t["artist_popularity"] = popularity
    t["mood"] = mood
    
    if genres:
        genre_counter.update(genres)
    playlist_moods.append(mood)

# 예시 출력 (앞 10곡)
for t in tracks[:10]:
    print(
        f"{t['name']} - {t['artists']} | "
        f"mood={t['mood']} | "
        f"genres={t['genres'][:2]} | "
        f"artist_pop={t['artist_popularity']}"
    )

print("\n=== Playlist mood summary ===")
print("Mood distribution:", Counter(playlist_moods))
print("Top 10 genres:", genre_counter.most_common(10))


It Ain't Necessarily So - Mary Lou Williams | mood=smooth | genres=['jazz', 'bebop'] | artist_pop=38
When Sunny Gets Blue - McCoy Tyner | mood=emotional | genres=['jazz', 'hard bop'] | artist_pop=54
Love Theme From Spartacus - Rudy Van Gelder Remaster / 2006 - Yusef Lateef | mood=smooth | genres=['jazz', 'hard bop'] | artist_pop=51
The Shadow of Your Smile - Dexter Gordon, Slide Hampton | mood=emotional | genres=['hard bop', 'jazz'] | artist_pop=49
So Tender - Keith Jarrett, Gary Peacock, Jack DeJohnette | mood=emotional | genres=['jazz', 'jazz ballads'] | artist_pop=54
In A Sentimental Mood - Duke Ellington, John Coltrane | mood=emotional | genres=['jazz', 'big band'] | artist_pop=62
Quiet Temple - Donald Byrd & Booker Little | mood=smooth | genres=['jazz'] | artist_pop=11
Dancing - bLAck pARty | mood=emotional | genres=['indie soul', 'alternative r&b'] | artist_pop=46
The Things We Did Last Summer - Freddie Hubbard | mood=emotional | genres=['hard bop', 'jazz'] | artist_pop=44
San Ju

In [14]:
pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.3 MB/s  0:00:04eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 2.1 MB/s  0:00:0036m-:--:--
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 2.4 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 2.5 MB/s  0:00:12m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [streamlit]14 [streamlit]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import streamlit as st

playlist_url = st.text_input("Spotify playlist URL")

if st.button("Analyze") and playlist_url:
    # 여기서 방금 만든 분석 함수 호출
    # summary, mood_distribution, genre_distribution, tracks 반환
    ...
    st.write(summary)
    st.bar_chart(...)
    st.dataframe(...)


2025-12-23 16:50:30.751 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-23 16:50:30.752 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-23 16:50:30.752 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-23 16:50:30.753 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-23 16:50:30.754 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-12-23 16:50:30.754 WARNING streamlit.runtime.scriptrunner_utils.script_run_c